# Proyecto ETL
#### Grupo 1:  Gledson - Eva - Sara - Rubén - Luis Q

### - - Importación de módulos y librerías - - 

In [ ]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver

from Scrappers.Scrapper_Comentarios import extrae_comentarios_meneo

# Extracción de datos

Se definen las funciónes auxiliares para la extracción

#### Bucle para el Scrapping de Comentarios  ####

In [ ]:
df_completo = pd.read_csv("Data/MeneosCompleto.csv", sep=";")

browser = webdriver.Chrome()

limite_comentarios = 1000
contador_comentarios = 0
lista_aux_comentarios = []

for meneo in df_completo["URL"]:
    if contador_comentarios < limite_comentarios:
        pagina_comentarios = 1

        browser.get(f"{meneo}/standard/")
        Soup_pagina_meneo = BeautifulSoup(browser.page_source, "html.parser")

        try:
            total_paginas_comentarios = int(Soup_pagina_meneo.find("div", class_="pages")["data-total-pages"])
        except:
            total_paginas_comentarios = 1

        while pagina_comentarios <= total_paginas_comentarios:
            if pagina_comentarios == 1:
                resultado = extrae_comentarios_meneo(Soup_pagina_meneo, pagina_comentarios)
            else:
                browser.get(f"{meneo}/standard/{pagina_comentarios}")
                Soup_pagina_meneo = BeautifulSoup(browser.page_source, "html.parser")
                resultado = extrae_comentarios_meneo(Soup_pagina_meneo, pagina_comentarios)
            
            lista_aux_comentarios.extend(resultado)
            contador_comentarios += len(resultado)
            pagina_comentarios += 1
    else:
        break
    
browser.close()

In [ ]:
df = pd.DataFrame(lista_aux_comentarios)

In [ ]:
df.to_csv(f'Data/{contador_comentarios}_Comentarios.csv', sep=';', encoding='utf-8', index=False)